In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, date_format
import re 

# --- INÍCIO: FUNÇÃO AUXILIAR SNAKE_CASE ---
def to_snake_case(name):
    """
    Converte uma string (CamelCase, UPPER_CASE, etc.) para snake_case.
    """
    # Insere _ antes de letras maiúsculas
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    # Insere _ antes de grupos de letras maiúsculas/números
    s2 = re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1)
    # Converte tudo para minúsculo
    return s2.lower()
# --- FIM: FUNÇÃO AUXILIAR SNAKE_CASE ---

# =======================================================================
# 1. CONFIGURAÇÃO E INICIALIZAÇÃO DO SPARK
# =======================================================================
conf = SparkConf()
conf.set(
    "spark.jars.packages",
    "org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.11.901"
)
conf.set(
    "spark.hadoop.fs.s3a.aws.credentials.provider",
    "com.amazonaws.auth.InstanceProfileCredentialsProvider"
)

spark = SparkSession.builder.appName("GoogleTwitterIntegration").config(conf=conf).getOrCreate()

# =======================================================================
# 2. DEFINIÇÃO DOS CAMINHOS DOS ARQUIVOS DE ENTRADA E SAÍDA
# =======================================================================

# Arquivos de Entrada no bucket trusted (SOMENTE GOOGLE E TWITTER)
INPUT_PATHS = {
    "google": "s3a://bucket-trusted-upa-connect-sofh/BasesExternas/Google/part-00000-61facf5d-5dac-44c5-bda2-eb82fc4c171b-c000.csv",
    "twitter": "s3a://bucket-trusted-upa-connect-sofh/BasesExternas/Twitter/sentimentos_encontrados.csv"
}

# Caminho de Saída no bucket client
FINAL_OUTPUT_DIR = "s3a://bucket-client-upa-connect-sofh/avaliacoes"
FINAL_FILENAME = "GoogleTwitterIntegrados.csv"
TEMP_STAGING_DIR = f"{FINAL_OUTPUT_DIR}/_temp_staging_gt"
FINAL_OUTPUT_PATH = f"{FINAL_OUTPUT_DIR}/{FINAL_FILENAME}"

# Coluna de Junção Comum (Padronizada para a Data apenas - YYYY-MM-DD)
JOIN_COLUMN = "DataRegistro"

# =======================================================================
# 2.1. EXCLUSÃO DO ARQUIVO FINAL ANTERIOR
# =======================================================================

print(f"Verificando e excluindo o arquivo final anterior em: {FINAL_OUTPUT_PATH}")
try:
    Path = spark._jvm.org.apache.hadoop.fs.Path
    hadoop_conf = spark._jsc.hadoopConfiguration()
    fs = Path(FINAL_OUTPUT_PATH).getFileSystem(hadoop_conf)

    if fs.exists(Path(FINAL_OUTPUT_PATH)):
        fs.delete(Path(FINAL_OUTPUT_PATH), False) 
        print(f"Arquivo anterior {FINAL_FILENAME} excluído com sucesso.")
    else:
        print(f"Arquivo anterior {FINAL_FILENAME} não encontrado. Prosseguindo.")

except Exception as e:
    print(f"\nATENÇÃO: Não foi possível verificar/excluir o arquivo final no S3. {e}")

# =======================================================================
# 3. LEITURA DOS DADOS E PREPARAÇÃO PARA JOIN (Delimitadores Corrigidos)
# =======================================================================

dataframes = {}

print("Iniciando a leitura e padronização dos DataFrames (Google e Twitter)...")
for name, path in INPUT_PATHS.items():
    print(f"Lendo {name} de: {path}")
    
    read_options = {
        "header": "true",
        "encoding": "UTF-8"
    }

    # CORREÇÃO DO DELIMITADOR: Google usa ';', Twitter usa ','
    if name == "google":
        read_options["delimiter"] = ";" 
    elif name == "twitter":
        read_options["delimiter"] = ","
        
    df = spark.read.options(**read_options).csv(path)
    
    # Padronização da Coluna de Data
    if name == "google":
        # Renomeia a coluna real 'Data_Publicacao_Avaliacao' para JOIN_COLUMN
        df = df.withColumnRenamed("Data_Publicacao_Avaliacao", JOIN_COLUMN)

    elif name == "twitter":
        # Renomeia a coluna real 'dataPublicacao' para JOIN_COLUMN
        df = df.withColumnRenamed("dataPublicacao", JOIN_COLUMN)
    
    # Renomeia colunas para evitar conflitos (exceto a coluna de junção)
    for c in df.columns:
        if c.upper() != JOIN_COLUMN.upper():
            df = df.withColumnRenamed(c, f"{name.upper()}_{c}")
            
    dataframes[name] = df
    print(f"Schema de {name} após preparação:")
    dataframes[name].printSchema()

print("Leitura e preparação concluídas.")

# =======================================================================
# 4. REALIZAÇÃO DA JUNÇÃO (FULL OUTER JOIN)
# =======================================================================
print("\nIniciando a junção dos DataFrames...")

df_google = dataframes["google"]
df_twitter = dataframes["twitter"]

# Junção usando a coluna DataRegistro, que agora existe nos dois DataFrames.
df_base = df_google.join(df_twitter, on=JOIN_COLUMN, how="full_outer")
print("Junção entre Google e Twitter concluída.")

# =======================================================================
# 5. REMOÇÃO DE COLUNAS DE ID E PADRONIZAÇÃO FINAL PARA SNAKE_CASE
# =======================================================================

# 1. REMOÇÃO DE COLUNAS DE ID
# Lista vazia, conforme observação de que os IDs não existem
COLUNAS_PARA_REMOVER = [] 

colunas_para_renomear = [c for c in df_base.columns if c not in COLUNAS_PARA_REMOVER]
df_base = df_base.select(colunas_para_renomear)

# 2. PADRONIZAÇÃO PARA SNAKE_CASE
print("\nPadronizando nomes das colunas para snake_case...")

col_rename_map = {c: to_snake_case(c) for c in df_base.columns}

for old_name, new_name in col_rename_map.items():
    df_base = df_base.withColumnRenamed(old_name, new_name)

df_base.printSchema()
print(f"Total de linhas na base integrada (Google/Twitter): {df_base.count()}")

# =======================================================================
# 6. SALVANDO E RENOMEANDO O RESULTADO NO S3
# =======================================================================

# 1. Escreve o resultado no caminho temporário
print(f"\nEscrevendo dados temporariamente em: {TEMP_STAGING_DIR}")

df_base.coalesce(1).write \
    .option('delimiter', ';') \
    .option('header', 'true') \
    .option('encoding', 'UTF-8') \
    .mode('overwrite') \
    .csv(TEMP_STAGING_DIR)

# 2. Renomeia o arquivo gerado
try:
    Path = spark._jvm.org.apache.hadoop.fs.Path
    hadoop_conf = spark._jsc.hadoopConfiguration()
    fs = Path(TEMP_STAGING_DIR).getFileSystem(hadoop_conf)

    list_status = fs.globStatus(Path(TEMP_STAGING_DIR + "/part-00000-*.csv"))

    if list_status:
        generated_file_path = list_status[0].getPath()
        final_output_path = Path(FINAL_OUTPUT_PATH)
        fs.rename(generated_file_path, final_output_path)
        fs.delete(Path(TEMP_STAGING_DIR), True) 
        
        print(f"\n✅ Base integrada salva e renomeada com sucesso para: {final_output_path}")

    else:
        print("\nErro: Não foi possível encontrar o arquivo CSV gerado (part-00000-*.csv) no caminho temporário.")

except Exception as e:
    print(f"\nOcorreu um erro durante a renomeação do arquivo no S3: {e}")

# Encerra a sessão Spark
spark.stop()

:: loading settings :: url = jar:file:/usr/local/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-3ed9a50e-cdf0-4bb3-af29-e8805f05fdb9;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 479ms :: artifacts dl 19ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.4 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	:: evicted modules:
	com.amazonaws#aws-java-sdk-bundle;1.11.901 by [com.amazonaws#aws-java-sdk-bundle;1.12.262] in [default]
	---------------------------------------------------------------------
	|     

Verificando e excluindo o arquivo final anterior em: s3a://bucket-client-upa-connect-sofh/avaliacoes/GoogleTwitterIntegrados.csv


25/11/02 00:21:02 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


Arquivo anterior GoogleTwitterIntegrados.csv não encontrado. Prosseguindo.
Iniciando a leitura e padronização dos DataFrames (Google e Twitter)...
Lendo google de: s3a://bucket-trusted-upa-connect-sofh/BasesExternas/Google/part-00000-61facf5d-5dac-44c5-bda2-eb82fc4c171b-c000.csv


Schema de google após preparação:
root
 |-- GOOGLE_Nome_Upa: string (nullable = true)
 |-- GOOGLE_Contagem_Avaliacoes: string (nullable = true)
 |-- GOOGLE_Pontuacao_Total: string (nullable = true)
 |-- GOOGLE_Cidade: string (nullable = true)
 |-- GOOGLE_Estado: string (nullable = true)
 |-- GOOGLE_Bairro: string (nullable = true)
 |-- GOOGLE_Rua: string (nullable = true)
 |-- GOOGLE_CEP: string (nullable = true)
 |-- GOOGLE_Origem_Avaliacao: string (nullable = true)
 |-- GOOGLE_Data_Coleta: string (nullable = true)
 |-- DataRegistro: string (nullable = true)
 |-- GOOGLE_Texto_Avaliacao: string (nullable = true)
 |-- GOOGLE_Estrelas: string (nullable = true)

Lendo twitter de: s3a://bucket-trusted-upa-connect-sofh/BasesExternas/Twitter/sentimentos_encontrados.csv
Schema de twitter após preparação:
root
 |-- TWITTER_palavra: string (nullable = true)
 |-- TWITTER_tipo: string (nullable = true)
 |-- DataRegistro: string (nullable = true)

Leitura e preparação concluídas.

Iniciando a junç

Total de linhas na base integrada (Google/Twitter): 3702

Escrevendo dados temporariamente em: s3a://bucket-client-upa-connect-sofh/avaliacoes/_temp_staging_gt


25/11/02 00:21:21 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
25/11/02 00:21:22 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.



✅ Base integrada salva e renomeada com sucesso para: s3a://bucket-client-upa-connect-sofh/avaliacoes/GoogleTwitterIntegrados.csv
